This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace.

## Step 1: Setup
Simply install the provided requirements.txt file by running the following command:
```bash
pip install -r requirements.txt
```

In [1]:
import os
def set_open_ai_api():
    os.environ["OPENAI_API_KEY"] =
    os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

You can now directly skip to step 4 since the fake listings have been saved for you and since each new run of the cells of step 2&3 would generate a new set of listings (reproducibility!).

## Step 2 & 3: Generating Fake Listings Using LLM and Saving Them

In [ ]:
import openai
import os

from langchain.memory import ConversationBufferMemory
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

from langchain.llms import OpenAI


In [ ]:
from langchain.prompts import PromptTemplate

listing_template = PromptTemplate(template=
"""Neighborhood: {neighborhood}
Price: ${price}
Bedrooms: {bathrooms}
Bathrooms: {bedrooms}
House Size: {house_size} sqft

Description: {description}

Neighborhood Description: {neighborhood_description}
""",
                                  input_variables=["neighborhood", "price", "bedrooms", "bathrooms",
                                                   "house_size", "description", "neighborhood_description"]
                                  )



In [ ]:
example_inputs = [
    {
        "neighborhood": "Downtown Abbey",
        "price": 300000,
        "bedrooms": 3,
        "bathrooms": 2,
        "house_size": 2000,
        "description": "A beautiful appartment with a large terrace overlooking the city's skyscrapers with 3 well-lit bedrooms and 2 bathrooms means you have enough room even for the occasional guest coming over to celebrate new year's with you :)",
        "neighborhood_description": "A vibrant neighborhood which is very centrally located and specially well connected public transport connections means you are close to everything you need for your everyday life from groceries to cafes and restaurants and shopping."
    },
    {
        "neighborhood": "City Heights",
        "price": 500000,
        "bedrooms": 4,
        "bathrooms": 3,
        "house_size": 3000,
        "description": "A cozy house with a wonderful garden overlooking the city from its hillside location",
        "neighborhood_description": "Located in the city outskirts, this neighborhood is known for its great views and quiet streets with a very family-friendly atmosphere. In no time you are in the midst of a beautiful forest where you can calm all your senses and breath in the cool fresh air. Sightings of deer and other wildlife are common."
    },
    {
        "neighborhood": "Green Oaks",
        "price": 800000,
        "bedrooms": 3,
        "bathrooms": 2,
        "house_size": 2000,
        "description": "A luxurious villa with a large garden and a swimming pool with a winter garden, 2 parking spots, and a garage as well as a large terrace and wonderful finishing.",
        "neighborhood_description": "Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze."
    }
]

In [ ]:
from langchain.prompts import FewShotPromptTemplate

few_shot_template = FewShotPromptTemplate(
    example_prompt=listing_template,
    input_variables=["input"],
    examples=example_inputs,
    suffix="Use the examples above to generate the following: {input}",
)

listing_seperator = "===="
instruction = "Generate {} listings of houses. Be creative in regards neighborhood_description, and description and keep it confined to three sentences each. Keep the prices between $125000 $1500000, the number of bedrooms under 6 and the number of bathrooms under 3 and the total area under 3500sqft. VERY IMPORTANT: Output the results in the same format as the examples keeping even the order of properties the same as in the examples. Add the characters {} before each new listing"
num_listings = 30
prompt_to_use = few_shot_template.format(input=instruction.format(num_listings, listing_seperator))



In [ ]:
print(prompt_to_use)

In [ ]:
from langchain.chat_models import ChatOpenAI

model_name = "gpt-3.5-turbo"
llm = OpenAI(model_name=model_name, temperature=0.0, max_tokens=3500)

In [ ]:
result = llm(prompt=prompt_to_use)

In [ ]:
print(f"LLM result:\n{result}")

In [ ]:
import pandas as pd
import locale
import re

# Set the locale to 'en_US.UTF-8' for parsing numbers with commas
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# Split the result into individual listings
listings = result.split(listing_seperator)[1:]
# print(f"Here are the listings in a list:\n{listings}")

# Define the columns based on input_variables
columns = ["neighborhood", "price", "bedrooms", "bathrooms", "house_size", "description", "neighborhood_description"]

# Parse the listings into a list of dictionaries
data = []
for listing in listings:
    print(f"Parsing listing: {listing[1:150]} ...\n")
    try:
        entry = {
            "neighborhood": re.search(r"^Neighborhood: (.+)$", listing, re.MULTILINE).group(1),
            "price": locale.atoi(re.search(r"^Price: \$(.+)$", listing, re.MULTILINE).group(1)),
            "bedrooms": int(re.search(r"^Bedrooms: (.+)$", listing, re.MULTILINE).group(1)),
            "bathrooms": int(re.search(r"^Bathrooms: (.+)$", listing, re.MULTILINE).group(1)),
            "house_size": locale.atoi(re.search(r"^House Size: (.+) sqft$", listing, re.MULTILINE).group(1)),
            "description": re.search(r"^Description: (.+)$", listing, re.MULTILINE).group(1),
            "neighborhood_description": re.search(r"^Neighborhood Description: (.+)$", listing, re.MULTILINE).group(1)
        }

        data.append(entry)
    except Exception as e:
        print("Error parsing listing")
        print(listing)
        print(f"Because of \n{e}")
        print("\n")
        continue

# Create a DataFrame
df = pd.DataFrame(data, columns=columns)

Let's view the dataframe and assert the generated input makes sense and finally save into a csv.

In [ ]:
df.head(n=num_listings)

In [ ]:
df.to_csv("listings.csv", index=False)


## Step 4: Basic Listings Recommendation without Using a Set of Relevant Listings

Feel free to start this step after step 1 directly

In [22]:
def ask_llm_and_print_question_and_response(chain,question):
    response=chain(inputs={"question": ("%s" % question)})
    print(f"The user asked: {question}")
    print(f"The smart assistant answered: {response['answer']}")
    return response

In [23]:
from langchain.llms import OpenAIChat
set_open_ai_api()
model="gpt-3.5-turbo"
llm = OpenAIChat(model_name=model,temperature=0.5)

In [24]:
# Hard-coding the user preferences using the ones provided in the project - with minor modifications - for reproducibility purposes wit

questions = [
    "How big do you want your house to be?"
    "What are 3 most important things for you in choosing this property?",
    "Which amenities would you like?",
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?",
]
answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

We create a conversation chain with memory and simulate the question - answer interaction such that we feed its memory with the questions and answers since running the interaction for real might not necessarily lead the llm to asking the right questions due to its unpredictability

In [25]:
from langchain.memory import ChatMessageHistory
def create_chat_message_history(questions, answers):
    chat_message_history=ChatMessageHistory()
    for question, answer in zip(questions, answers):
        chat_message_history.add_ai_message(question)
        chat_message_history.add_user_message(answer)
    return chat_message_history

In [30]:
base_system_prompt = """You are a real estate agent. You have a new client who is looking for a new home.

The client has provided answers to important questions in the context.

Proceed as follows:
- First ask the user if he/she is ready to receive recommendations.
- If he/she enters says 'yes' DON'T MAKE a recommednation yet. First summarize your understanding of the user preferences and ask the user if he/she is ready to receive recommendations telling the user to either type 'yes' or 'no'
- If he types 'yes' you match the user preferences with listings and make suggestions."""

In [31]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

memory=ConversationBufferMemory(conversation_history=create_chat_message_history(questions,answers), llm=llm, chat_memory=create_chat_message_history(questions,answers),memory_key="context")

qa_template = PromptTemplate(template=
                             ("""%s

Context: {context}

Question: {question}
""" % base_system_prompt),
                             input_variables=["context", "question"]
                             )
qa_chain = ConversationChain(llm=llm,  memory=memory, prompt=qa_template,input_key="question",output_key="answer", verbose=False)


In [32]:
question = "I am waiting for your recommendations"
ask_llm_and_print_question_and_response(qa_chain,question=question)

The user asked: I am waiting for your recommendations
The smart assistant answered: Are you ready to receive recommendations based on your preferences? Please type 'yes' or 'no'.


{'question': 'I am waiting for your recommendations',
 'context': 'AI: How big do you want your house to be?What are 3 most important things for you in choosing this property?\nHuman: A comfortable three-bedroom house with a spacious kitchen and a cozy living room.\nAI: Which amenities would you like?\nHuman: A quiet neighborhood, good local schools, and convenient shopping options.\nAI: Which transportation options are important to you?\nHuman: A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.\nAI: How urban do you want your neighborhood to be?\nHuman: Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.',
 'answer': "Are you ready to receive recommendations based on your preferences? Please type 'yes' or 'no'."}

In [33]:
question = "yes"
response=ask_llm_and_print_question_and_response(qa_chain,question=question)

The user asked: yes
The smart assistant answered: Great! Based on your preferences, I understand that you are looking for a comfortable three-bedroom house with a spacious kitchen and a cozy living room. You value a quiet neighborhood, good local schools, and convenient shopping options. Additionally, you are looking for a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. You also prefer easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.

Are you ready to receive recommendations based on these preferences? Please type 'yes' or 'no'.


In [34]:
question = "yes"
response=ask_llm_and_print_question_and_response(qa_chain,question=question)

The user asked: yes
The smart assistant answered: Great! Based on your preferences, I have found a few listings that match what you are looking for. Here are some recommendations:

1. Beautiful 3-bedroom house in a quiet neighborhood with a spacious kitchen, cozy living room, and backyard for gardening. Features a two-car garage and modern, energy-efficient heating system. Close to good local schools and convenient shopping options. Easy access to a reliable bus line and bike-friendly roads.

2. Charming house with 3 bedrooms, a large kitchen, and a comfortable living room. Located in a peaceful neighborhood with good local schools and convenient shopping options nearby. Includes a two-car garage, backyard for gardening, and modern heating system. Close to a reliable bus line and major highway.

3. Cozy 3-bedroom home in a quiet area with a spacious kitchen and inviting living room. Features a backyard for gardening, two-car garage, and energy-efficient heating system. Close to good lo

**We can see the result above is totally fake. Let's see what we can do about this using our generated listings and vector db'**

## Step 5: Using the Generated Listings to Provide Real Context

In [35]:
from typing import List, Dict, Any
from langchain.memory import ConversationBufferMemory

# there is a solution to ConversationChain problem of not being able to use more than one input:
# The ConversationBufferMemory is extended to be able to pass other information.
# Thanks to https://github.com/langchain-ai/langchain/issues/1800#issuecomment-1598128244
class ExtendedConversationBufferMemory(ConversationBufferMemory):
    extra_variables:List[str] = []

    @property
    def memory_variables(self) -> List[str]:
        """Will always return list of memory variables."""
        return [self.memory_key] + self.extra_variables

    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """Return buffer with history and extra variables"""
        d = super().load_memory_variables(inputs)
        d.update({k:inputs.get(k) for k in self.extra_variables})
        return d

In [36]:
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate


memory_with_listings=ExtendedConversationBufferMemory(
    conversation_history=create_chat_message_history(questions,answers),
    llm=llm,
    chat_memory=create_chat_message_history(questions,answers),
    memory_key="context",
    extra_variables=["listings"])


qa_template_with_listings=PromptTemplate(template=
                          (("""%s. Only provide suggestions from the available listings. '

Context: {context}

Listings: {listings}

Question: {question}
""") % base_system_prompt),
                          input_variables=["context", "listings", "question"]
                          )



qa_chain_with_listings = ConversationChain(llm=llm,  memory=memory_with_listings, prompt=qa_template_with_listings,input_key="question" ,output_key="answer", verbose=False)



In [37]:
response=ask_llm_and_print_question_and_response(qa_chain_with_listings,question="I am waiting for your recommendations")
preference_summary_by_smart_assistant=response['answer']

The user asked: I am waiting for your recommendations
The smart assistant answered: First, may I summarize your preferences? You are looking for a comfortable three-bedroom house with a spacious kitchen and a cozy living room. Your top priorities are a quiet neighborhood, good local schools, and convenient shopping options. You also value having a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. In terms of transportation, you prefer easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. Is that correct?

Are you ready to receive recommendations based on your preferences? Please type 'yes' or 'no'.


Now we use the preference summary by the smart assistant to retreive suitable listings from the vector db

In [38]:
from langchain.document_loaders import CSVLoader
import random

set_open_ai_api()
listings_loader = CSVLoader("listings.csv")
listing_documents=listings_loader.load()
print(f"There are {len(listing_documents)} listings loaded")
sample_listings = random.sample(listing_documents, 2)
print(f"Here are two sample loaded listings:\n{sample_listings}")

There are 28 listings loaded
Here are two sample loaded listings:
[Document(page_content='neighborhood: Riverbend Estates\nprice: 650000\nbedrooms: 4\nbathrooms: 2\nhouse_size: 2300\ndescription: Elegant home with a grand entrance and high ceilings, featuring a gourmet kitchen and luxurious master suite. The backyard oasis includes a patio, garden, and fire pit for outdoor entertaining and relaxation.\nneighborhood_description: Riverbend Estates is a prestigious neighborhood known for its upscale homes and scenic views of the river. Residents enjoy access to walking trails, community events, and upscale dining options, as well as proximity to top-rated schools and shopping centers.', metadata={'source': 'listings.csv', 'row': 17}), Document(page_content='neighborhood: Hillside Manor\nprice: 175000\nbedrooms: 2\nbathrooms: 1\nhouse_size: 1500\ndescription: Charming cottage-style home with a cozy fireplace and a private backyard oasis with a deck and garden. The bright and airy living sp

In [39]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

embeddings_model=OpenAIEmbeddings()
# I found no better way to ensure the chromadb is reset other than running twice with first call including delete_collection()
# Without these, each time the cell is rerun the documents are added all over new leading to wrong results in k-nearest similarity search
listings_db=Chroma.from_documents(documents=listing_documents,embedding=embeddings_model).delete_collection()
listings_db=Chroma.from_documents(documents=listing_documents,embedding=embeddings_model)

In [40]:
listings_db._collection.count()

28

In [41]:
most_similar_documents=listings_db.similarity_search(query=preference_summary_by_smart_assistant,k=4)
print(f"Most similar documents to the user preferences: {most_similar_documents}")

Most similar documents to the user preferences: [Document(page_content='neighborhood: Cedarwood Village\nprice: 225000\nbedrooms: 3\nbathrooms: 2\nhouse_size: 1600\ndescription: Cozy home with a spacious backyard and deck for outdoor gatherings and relaxation. The open floor plan and large windows create a bright and inviting atmosphere throughout the house.\nneighborhood_description: Cedarwood Village is a family-friendly neighborhood with tree-lined streets and nearby parks for outdoor activities. Close to schools, shopping centers, and restaurants, making it a convenient and welcoming place to call home.', metadata={'row': 24, 'source': 'listings.csv'}), Document(page_content='neighborhood: Willow Creek\nprice: 425000\nbedrooms: 4\nbathrooms: 2\nhouse_size: 2000\ndescription: Spacious two-story home with a large backyard and deck for outdoor gatherings and relaxation. The updated kitchen and cozy living room provide a comfortable and inviting space for families to enjoy.\nneighborho

In [42]:
most_similar_documents_string="\n\n---\n".join([most_similar_document.page_content for most_similar_document in most_similar_documents])
print(most_similar_documents_string)

neighborhood: Cedarwood Village
price: 225000
bedrooms: 3
bathrooms: 2
house_size: 1600
description: Cozy home with a spacious backyard and deck for outdoor gatherings and relaxation. The open floor plan and large windows create a bright and inviting atmosphere throughout the house.
neighborhood_description: Cedarwood Village is a family-friendly neighborhood with tree-lined streets and nearby parks for outdoor activities. Close to schools, shopping centers, and restaurants, making it a convenient and welcoming place to call home.

---
neighborhood: Willow Creek
price: 425000
bedrooms: 4
bathrooms: 2
house_size: 2000
description: Spacious two-story home with a large backyard and deck for outdoor gatherings and relaxation. The updated kitchen and cozy living room provide a comfortable and inviting space for families to enjoy.
neighborhood_description: Willow Creek is a family-friendly neighborhood with top-rated schools and community parks for children to play and explore. Close to shop

Finally, we hit the LLM now with the extended context involving the most similar documents found from the vector db

In [43]:
response=qa_chain_with_listings(inputs={"question":"yes","listings":most_similar_documents_string})

In [44]:
print(f"The smart assistant answered: {response['answer']}")

The smart assistant answered: Yes


Let's verify if the top listing indeed exists

In [46]:
import pandas as pd
df=pd.read_csv("listings.csv")
filtered_df=df[df["neighborhood"]=="Meadowview Park"]
filtered_df.head()

,neighborhood,price,bedrooms,bathrooms,house_size,description,neighborhood_description
26,Meadowview Park,700000,5,2,2500,Spacious two-story home with a large backyard ...,Meadowview Park is a family-friendly neighborh...


**q.e.d. Good job :)**

# Step 6: Personalized Response

The assignment requires that the LLM is used to give a response of suitable listings wherein the listing is described in such a way such that the aspects of relevance to the user are highlighted

In [223]:
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate


memory_with_listings=ExtendedConversationBufferMemory(
    llm=llm,
    chat_memory=create_chat_message_history(questions,answers),
    memory_key="context",
    extra_variables=["listings"]) # resetting the memory with just the questions/answers

# We do the personalization of the listing response by instructing the LLM to do so
qa_template_with_listings_and_personalization=PromptTemplate(template=
(("""%s. Only provide suggestions from the available listings.

Context: {context}

Listings: {listings}

Question: {question}
--------
when you are recommending listings, craft a response in which a listing's **description** and **neighborhood description** are modified creatively from the original ones while preserving correctness. You should highlight aspects of relevance to the user's preferences as summarized by the AI.
""") % base_system_prompt),
                                         input_variables=["context", "listings", "question"]
                                         )



qa_chain_with_listings_and_personalization = ConversationChain(
    llm=llm,
    memory=memory_with_listings,                                                   prompt=qa_template_with_listings_and_personalization,
    input_key="question",
    output_key="answer",
    verbose=False)



In [224]:
response=ask_llm_and_print_question_and_response(qa_chain_with_listings_and_personalization,question="I am waiting for your recommendations")


The user asked: I am waiting for your recommendations
The smart assistant answered: I see that you are looking for a comfortable three-bedroom house with a spacious kitchen and a cozy living room in a quiet neighborhood with good local schools and convenient shopping options. You also mentioned wanting a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. Additionally, you prefer easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.

Are you ready to receive recommendations? Please type 'yes' or 'no'.


In [225]:
preference_summary_by_smart_assistant=response['answer']

In [226]:
print(f"{preference_summary_by_smart_assistant=}")

preference_summary_by_smart_assistant="I see that you are looking for a comfortable three-bedroom house with a spacious kitchen and a cozy living room in a quiet neighborhood with good local schools and convenient shopping options. You also mentioned wanting a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. Additionally, you prefer easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.\n\nAre you ready to receive recommendations? Please type 'yes' or 'no'."


In [227]:
most_similar_documents=listings_db.similarity_search(query=preference_summary_by_smart_assistant,k=4)
most_similar_documents_string="\n\n---\n".join([most_similar_document.page_content for most_similar_document in most_similar_documents])

In [228]:
response=qa_chain_with_listings_and_personalization(inputs={"question":"yes","listings":most_similar_documents_string})
print(f"Here's the final response by the assistant which should have relevant listings rephrased to highlight relevant aspects of the description to the user's preferences:\n"
      f"{response['answer']}")

Here's the final response by the assistant which should have relevant listings rephrased to highlight relevant aspects of the description to the user's preferences:
Great! Based on your preferences for a comfortable three-bedroom house with a spacious kitchen and a cozy living room in a quiet neighborhood with good local schools and convenient shopping options, as well as a backyard for gardening, a two-car garage, and a modern, energy-efficient heating system, I have some listings for you:

1. Listing in Oakridge Park:
Price: $375,000
Bedrooms: 3
Bathrooms: 2
House Size: 1900 sqft
Description: Charming home with a beautifully landscaped backyard perfect for gardening enthusiasts. The open-concept kitchen and living room provide a warm and inviting space for families to relax and entertain.
Neighborhood Description: Oakridge Park is a serene and family-friendly neighborhood with excellent schools and easy access to shopping centers and dining options. It's a peaceful retreat for those 

Now it is time to match the provided listings with the person's preferences and comparing to the original descriptions!

First we review the original questions and user's answers

In [229]:
for question, answer in zip(questions,answers):
    print(f"Q: {question}")
    print(f"a: {answer}")

Q: How big do you want your house to be?What are 3 most important things for you in choosing this property?
a: A comfortable three-bedroom house with a spacious kitchen and a cozy living room.
Q: Which amenities would you like?
a: A quiet neighborhood, good local schools, and convenient shopping options.
Q: Which transportation options are important to you?
a: A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.
Q: How urban do you want your neighborhood to be?
a: Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.


As we see, the emphasis is on quiteness of neighborhood and having good schools with proximity to shopping options menioned last. Also the ability to do gardening and a good connection to public transport and roads


Here's the description of the top match as recommended by the smart agent:

"""

Description: Charming home with a beautifully landscaped backyard perfect for gardening enthusiasts. The open-concept kitchen and living room provide a warm and inviting space for families to relax and entertain.

Neighborhood Description: Oakridge Park is a serene and family-friendly neighborhood with excellent schools and easy access to shopping centers and dining options. It's a peaceful retreat for those seeking a quiet yet convenient lifestyle.
"""

Finally let's retrieve the original description and neighborhood description of the match

In [231]:

neighborhood_top_listing = "Oakridge Park"
assert len(df[df["neighborhood"] == neighborhood_top_listing]) == 1 # to justify fetching only first row with this neighborhood
top_listing_from_df = df[df['neighborhood'] == neighborhood_top_listing].iloc[0]
print(f"The original description of the top listing:\n{top_listing_from_df['description']}")
print(f"The original neighborhood description of the top listing:\n{top_listing_from_df['neighborhood_description']}")

The original description of the top listing:
Traditional home with a spacious backyard and deck for outdoor entertaining and relaxation. The cozy living room and updated kitchen provide a comfortable and inviting space for families to enjoy.
The original neighborhood description of the top listing:
Oakridge Park is a quiet and friendly neighborhood with well-maintained parks and playgrounds for children to play. Close to shopping centers, restaurants, and schools, making it a convenient and desirable place to live for families and young professionals. 


Observing the details we notice, that the AI generated listing
- Maintains references to coziness and family-friendliness while creatively rephrasing the description.
- Maintains references to proximity to schools, shopping as well as restaurants (calling it 'dining' as opposed to 'restaurants' in the original description) without adding anything else not mentioned
- Although the user's preferences mentioned access to public transport and proximity to highway, the LLM does not generate a fake description in this regard maintaining correctness.


**Done! :)**